In [ ]:
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
from ddeint import ddeint  # SciPy-based DDE solver:contentReference[oaicite:1]{index=1}

# Model parameters
k_cat, k_leak, k_rec, k_in = 20.0, 0.5, 0.1, 0.1
nu = 1.0
k_form, k_degS, k_degM = 5.0, 10.0, 1.0
n = 4
K, Ks = 0.5, 0.3
tau_p, tau_f, tau_d = 15.0, 0.0, 15.0

# Hill functions
def H(x): 
    return x**n / (K**n + x**n)
def Hs(x):
    return x**n / (Ks**n + x**n)

# DDE system definition
def model(Y, t):
    A2, S, M, A = Y(t)                       # current values
    M_delay = Y(t - tau_p)[2]                # M(t - tau_p)
    S_feed  = Y(t - tau_f)[1]                # S(t - tau_f) (immediate since tau_f=0)
    M_decay = Y(t - tau_d)[2]                # M(t - tau_d)
    # DDEs:
    dA2dt = k_in + k_rec * A - k_cat * A2 * H(M_delay) - k_leak * A2
    dSdt  = (k_cat/nu)*A2 * H(M_delay) + (k_leak/nu)*A2 - k_form * Hs(S) - k_degS * S
    dMdt  = k_form * Hs(S_feed) - k_degM * M_decay
    dAdt  = k_cat * A2 * H(M_delay) + k_leak * A2 + k_degS * S - k_rec * A
    return np.array([dA2dt, dSdt, dMdt, dAdt])

# History function (initial values for t <= 0)
A2_0, S_0, M_0, A_0 = 0.0, 0.0, 0.01, 0.0
g = lambda t: np.array([A2_0, S_0, M_0, A_0])

# Time span for simulation (0 to 800 units, with 0.1-unit steps for output)
t_max = 800
t_eval = np.linspace(0, t_max, 8001)

# Integrate the DDEs
sol = ddeint(model, g, t_eval)

# Extract the sum S+M
S_plus_M = sol[:,1] + sol[:,2]

# Plot the results
import matplotlib.pyplot as plt
plt.figure(figsize=(8,4))
plt.plot(t_eval, S_plus_M, color='b')
plt.xlabel("Time")
plt.ylabel("Concentration (S + M)")
plt.title("Time-course of S+M (Intermediate + Autocatalyst)")
plt.show()


ModuleNotFoundError: No module named 'ddeint'